# Ideal Diode Example

The simplest way to model a solar cell is as an **ideal diode**, described by the following equation:
$$J(V,T) = J_L+J_0(\exp{\frac{qV}{nkT}}-1)$$
where by convention, $J_L$ (the light current) is negative and $J_0$ (the leakage current) is positive. $J_L$ is typically directly proportional to light intensity, which we will treat as fixed for this example, leaving us with two parameters to fit for: $J_0$ and the ideality factor $n$.

In [12]:
import numpy as np
import sys
import math
import pandas as pd
sys.path.append('../')
import bayesim.model as bym
import bayesim.param_list as byp

## List the parameters
First, we list our fitting parameters. We'll space $J_0$ logarithmically and $n$ linearly...

In [2]:
pl = byp.param_list()
pl.add_fit_param(name='J0',val_range=[1e-12,1e-7],length=20,spacing='log',units='mA/cm^2')
pl.add_fit_param(name='n',val_range=[1,2])

We can inspect the `param_list` object and see what kind of information is inside...

In [3]:
pl.fit_params

[{'edges': array([  1.00000000e-12,   1.77827941e-12,   3.16227766e-12,
           5.62341325e-12,   1.00000000e-11,   1.77827941e-11,
           3.16227766e-11,   5.62341325e-11,   1.00000000e-10,
           1.77827941e-10,   3.16227766e-10,   5.62341325e-10,
           1.00000000e-09,   1.77827941e-09,   3.16227766e-09,
           5.62341325e-09,   1.00000000e-08,   1.77827941e-08,
           3.16227766e-08,   5.62341325e-08,   1.00000000e-07]),
  'length': 20,
  'min_width': 1.0592537251772889,
  'name': 'J0',
  'spacing': 'log',
  'units': 'mA/cm^2',
  'val_range': [1e-12, 1e-07],
  'vals': array([  1.33352143e-12,   2.37137371e-12,   4.21696503e-12,
           7.49894209e-12,   1.33352143e-11,   2.37137371e-11,
           4.21696503e-11,   7.49894209e-11,   1.33352143e-10,
           2.37137371e-10,   4.21696503e-10,   7.49894209e-10,
           1.33352143e-09,   2.37137371e-09,   4.21696503e-09,
           7.49894209e-09,   1.33352143e-08,   2.37137371e-08,
           4.21696503e

Note that there are both `edges` and `vals` computed - the `vals` will be the values at which computations will be done, while `edges` will be the edges of the grid, used in subdivision steps later on.

Finally, we define the measurement conditions. We won't be varying light intensity here, so we only need voltage and temperature.

In [5]:
ec = ['V','T']

## Attach experimental data
This is the next step that needs to be done before adding the model as it will use the experimental conditions we enumerate here to compute simulated outputs. For our "experimental" data here, we'll actually just simulate some ideal diodes so that we can directly see how good the Bayesian approach is at recovering true underlying parameters.

We'll use $n=1.37$ and $J_0=6.3\times 10^{-11}$ as our "true" parameters.

In [38]:
def compute_ID(ec,params):
    J_L = -0.03 # treat as fixed for now
    V = ec['V']
    T = ec['T']
    V_th = 8.61733e-5 * T # thermal voltage, kT/q
    n = params['n']
    J0 = params['J0']
    return J_L + J0*(math.exp(V/V_th/n)-1)

V = np.arange(-0.5,1.0,0.1)
T = np.arange(150,301,50)
params = {'n':1.37,'J0':6.3e-11}

data = []
for t in T:
    for v in V:
        data.append([v,t,compute_ID({'V':v,'T':t},params)])
        
data = pd.DataFrame.from_records(data=data,columns=['V','T','J'])
data.to_hdf('obs_data.h5','data',format='table',append=False)

In [39]:
#pd.read_hdf('obs_data.h5','data')
import deepdish as dd
print(dd.io.load('obs_data.h5')['data'])

               V    T             J
0  -5.000000e-01  150 -3.000000e-02
1  -4.000000e-01  150 -3.000000e-02
2  -3.000000e-01  150 -3.000000e-02
3  -2.000000e-01  150 -3.000000e-02
4  -1.000000e-01  150 -3.000000e-02
5  -1.110223e-16  150 -3.000000e-02
6   1.000000e-01  150 -2.999998e-02
7   2.000000e-01  150 -2.999494e-02
8   3.000000e-01  150 -2.856556e-02
9   4.000000e-01  150  3.765664e-01
10  5.000000e-01  150  1.152036e+02
11  6.000000e-01  150  3.266078e+04
12  7.000000e-01  150  9.257094e+06
13  8.000000e-01  150  2.623750e+09
14  9.000000e-01  150  7.436527e+11
15 -5.000000e-01  200 -3.000000e-02
16 -4.000000e-01  200 -3.000000e-02
17 -3.000000e-01  200 -3.000000e-02
18 -2.000000e-01  200 -3.000000e-02
19 -1.000000e-01  200 -3.000000e-02
20 -1.110223e-16  200 -3.000000e-02
21  1.000000e-01  200 -3.000000e-02
22  2.000000e-01  200 -2.999970e-02
23  3.000000e-01  200 -2.997923e-02
24  4.000000e-01  200 -2.856556e-02
25  5.000000e-01  200  6.908766e-02
26  6.000000e-01  200  6.814